In [ ]:
    import os
    import pandas as pd
    import numpy as np
    import argparse
    import random
    import sklearn
    import sklearn.metrics as metrics
    from sklearn.metrics import confusion_matrix
    #set the matplotlib backend so plots can be saved in the background
    #necessary since running on Google Cloud  
    import matplotlib
    %matplotlib inline
    matplotlib.use('Agg') 
    import matplotlib.pyplot as plt
    
    #Utilize GPUs
    import tensorflow as tf
    #Setup Keras 
    from keras.models import Sequential, Model
    from keras.layers.convolutional import Conv2D, MaxPooling2D
    from keras.layers.core import Activation, Flatten, Dense
    from keras.layers import BatchNormalization, Dropout, LeakyReLU
    from keras.optimizers import Adam, SGD, Adagrad
    from keras import backend as K 
    K.tensorflow_backend._get_available_gpus()
    
    from keras.preprocessing.image import ImageDataGenerator    
    from keras.callbacks import History
    
    #Setup VGG16
    from keras.applications import vgg16
    from keras.preprocessing import image
    from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

In [ ]:
print(K.tensorflow_backend._get_available_gpus())

In [ ]:
def count(filename, numberofimages = 1462):

    global classes
    global number_in_class
    classes = []
    number_in_class = []

    path, dirs, files = next(os.walk("../" + filename + "/"))  

    for dir in dirs:
        path2, dirs2, files2 = next(os.walk("../" + filename + "/" + dir))  
        classes.append(dir)
        number_in_class.append(len(files2)-1)

    print(number_in_class)
    global number_training
    global number_tested
    number_training = sum(number_in_class)
    number_tested = numberofimages - number_training
    return classes  
    return number_in_class
    return number_training
    return number_tested

def graph(xlabel, ylabel, classes, number_in_class):   
    df = pd.DataFrame({xlabel:classes, ylabel:number_in_class})
    df = df.sort_values([ylabel], ascending=False)
    df.plot(kind='bar', x = xlabel, y = ylabel, legend=False, 
            color=['tab:blue','tab:orange', 'tab:green', 'tab:red', 
                   'tab:purple', 'tab:brown', 'tab:pink',
                   'tab:olive', 'tab:cyan'], width = 0.95)

In [ ]:
count('boats_train_test/train')

In [ ]:
def build(num_classes, input_size, learnrate, EPOCHS, pad):
     
    K.clear_session()
    model = Sequential()

    #Convolution Layer
    model.add(Conv2D(32, (3, 3), padding = pad, input_shape = (56,56,3)))
    model.add(LeakyReLU(alpha = 0.1))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    
    #A Second Convolutional Layer
    model.add(Conv2D(64, (3, 3), padding = pad))
    model.add(LeakyReLU(alpha = 0.1))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    
    #Third Convolutional Layer
    model.add(Conv2D(128, (3, 3), padding = pad))
    model.add(LeakyReLU(alpha = 0.1))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(BatchNormalization())

    #Fully Connected Layer
    model.add(Flatten())
    #Xavier initialization of the weights 
    model.add(Dense(units = 512, kernel_initializer = 'glorot_normal'))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units = num_classes))
    model.add(Activation('softmax'))
    
    #Compiling the CNN
    print('[INFO] compiling model...')
    opt = Adam(lr = learnrate, decay = learnrate / EPOCHS)
    model.compile(optimizer = opt, loss = 'categorical_crossentropy',
                       metrics = ['accuracy'])
    print(model.summary())
    return model

    
def fit_model(dataset, learnrate, EPOCHS, batch, height = 56, width = 56):

    train_datagen = ImageDataGenerator(rescale = 1./255,
                                       shear_range = 0.2,
                                       zoom_range = 0.2,
                                       horizontal_flip = True)

    test_datagen = ImageDataGenerator(rescale = 1./255)

    training_set = train_datagen.flow_from_directory(dataset + '/train', target_size = (height, width), batch_size = batch, class_mode = 'categorical')

    test_set = test_datagen.flow_from_directory(dataset + '/test', target_size = (height, width), batch_size = batch, class_mode = 'categorical')
    
    #count(dataset + '/train')
    #num_classes = len(classes)
    #model = build(num_classes, learnrate, EPOCHS, input_size = (height, width))
    print('[INFO] training network')
    history = model.fit_generator(training_set, steps_per_epoch = number_training, epochs = EPOCHS, validation_data = test_set, validation_steps = number_tested)
    
    print("[INFO] serializing network...")

    #H = model.save(args["model"])
    
    # Plot training & validation accuracy values
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.savefig('AccuracyResult.png')
    plt.close()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.savefig('LossResult.png')
    plt.close()
    

In [ ]:
%matplotlib inline
for d in ['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3']:
    with tf.device(d):
        num_classes = len(classes)
        model = build(num_classes, learnrate = 0.0001, EPOCHS = 5, pad = 'valid', input_size = (56, 56, 3))
        fit_model(dataset = '../boats_train_test', learnrate = 0.0001, EPOCHS = 5, batch = 8)


In [ ]:
    #Could not figure out how to resolve errors when below code was in fit_model()
    
    #Sklearn metrics and Confusion matrix
    predictions = model.predict(test_set, steps = number_tested)
    # Get most likely class
    predicted_classes = np.argmax(predictions, axis=1)
    #Correct classes
    true_classes = test_set.classes
    #class_labels = list(test_set.class_indices.keys())
    #metrics
    report = metrics.classification_report(true_classes, predicted_classes)
    print(report) 
    #confusion matrix
    confusion_matrix = metrics.confusion_matrix(y_true = true_classes, y_pred = predicted_classes)
    print(confusion_matrix)
 

In [ ]:
def initvgg(train_dir, batch):
    K.clear_session()
    #Initialize the VGG model without the Fully Connected Layers, 
    #thus we can train the model again on our dataset
    global vgg16_model
    vgg16_model = VGG16(include_top = False, weights = 'imagenet',
                   input_shape = (224, 224, 3))
    
    height = 224
    width = 224

    #train_datagen = ImageDataGenerator(rescale = 1./255,
                                       #shear_range = 0.2,
                                       #zoom_range = 0.2,
                                       #horizontal_flip = True)
    
    #train_datagen =  ImageDataGenerator(preprocessing_function = preprocess_input, 
     #                               rotation_range = 90, horizontal_flip = True,
    #                              vertical_flip = True)
    
    #train_generator = train_datagen.flow_from_directory(train_dir, target_size = (height, width), batch_size = batch)
    

def build_transfer_model(vgg16_model, dropout, fc_layers, num_classes):
    for layer in vgg16_model.layers:
        layer.trainable = False

    x = vgg16_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    transfer_model = Model(inputs=vgg16_model.input, outputs=predictions)

    return transfer_model

def fit_transfer_model(dataset, EPOCHS, batch, FC_layers = [512], dropout = 0.5, learnrate = 0.00001):

    #count(dataset + '/train')
    EPOCHS = EPOCHS
#    transfer_model = build_transfer_model(vgg16_model, 
#                                      dropout=dropout, 
#                                      fc_layers=FC_layers, 
#                                      num_classes=len(classes))

    adam = Adam(lr=learnrate)
    transfer_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

    #filepath="./checkpoints/" + "ResNet50" + "_model_weights.h5"
    #checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
    #callbacks_list = [checkpoint]
    train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

    training_set = train_datagen.flow_from_directory(dataset + '/train', target_size = (224, 224), batch_size = batch, class_mode = 'categorical')
    
    test_datagen = ImageDataGenerator(rescale = 1./255)

    test_set = test_datagen.flow_from_directory(dataset + '/test', target_size = (224, 224), batch_size = batch, class_mode = 'categorical')

    
    print('[INFO] training network')
    history = transfer_model.fit_generator(training_set, epochs = EPOCHS, workers=8, steps_per_epoch = number_training // batch, validation_data = test_set, validation_steps = number_tested // batch, shuffle=True)
    
    print("[INFO] serializing network...")
    #global H
    #H = model.save(args["model"])
    
    
def evaluate_transfer_model():    
    # Plot training & validation accuracy values
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.savefig('AccuracyResult.png')
    plt.close()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.savefig('LossResult.png')
    plt.close()
    


In [ ]:
%matplotlib inline
for d in ['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3']:
    with tf.device(d):
        initvgg(train_dir = '../boats_train_test/train', batch = 8)
        transfer_model = build_transfer_model(vgg16_model, dropout=0.5, fc_layers=[512], num_classes=len(classes))
        fit_transfer_model(dataset = '../boats_train_test', EPOCHS = 5, batch = 8)
        evaluate_transfer_model()

In [ ]:
%matplotlib inline
initvgg(train_dir = '../boats_train_test/train', batch = 8)
transfer_model = build_transfer_model(vgg16_model, dropout=0.5, fc_layers=[512], num_classes=len(classes))
history = fit_transfer_model(dataset = '../boats_train_test', EPOCHS = 5, batch = 8, learnrate = 0.0001)
evaluate_transfer_model()

In [ ]:
#Could not figure resolve issues with code below when in evaluate_transfer_model()

# plot the training loss and accuracy
    plt.style.use("ggplot")
    plt.figure()
    N = EPOCHS
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
    plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
    plt.title("VGG16 Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig('VGGresults.png')
    
    #Sklearn metrics and Confusion matrix
    predictions = model.predict_generator(test_set, steps = number_tested)
    # Get most likely class
    predicted_classes = numpy.argmax(predictions, axis=1)
    #Correct classes
    true_classes = test_set.classes
    #class_labels = list(test_set.class_indices.keys())
    #metrics
    report = metrics.classification_report(true_classes, predicted_classes)
    print(report) 
    #confusion matrix
    confusion_matrix = metrics.confusion_matrix(y_true = true_classes, y_pred = predicted_classes)
    print(confusion_matrix)

